In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [3]:
from skimage.io import imread,imshow
from skimage.transform import resize
from sklearn.utils import shuffle
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPool2D,BatchNormalization,Dropout,Flatten,Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
%matplotlib inline

In [4]:
from skimage.io import imread,imshow
from skimage.transform import resize
from sklearn.utils import shuffle
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPool2D,BatchNormalization,Dropout,Flatten,Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
%matplotlib inline

In [5]:
# my Contribution: Reduce Dataset 

train_dataset_0_all=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/all/*.bmp')
train_dataset_0_hem=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/hem/*.bmp')
train_dataset_1_all=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/all/*.bmp')
train_dataset_1_hem=glob.glob('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/hem/*.bmp')

In [6]:
valid_data=pd.read_csv('../input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data_labels.csv')

In [7]:
Test_dataset = glob.glob('../input/leukemia-classification/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data')

In [8]:
a,b=len(train_dataset_0_all),len(train_dataset_1_all)
d=a+b
print('count:',d)

In [9]:
valid_data.head()

In [10]:
A=[]
H=[]
A.extend(train_dataset_0_all)
A.extend(train_dataset_1_all)
H.extend(train_dataset_0_hem)
H.extend(train_dataset_1_hem)
print(len(A))
print(len(H))

In [11]:
A=np.array(A)
H=np.array(H)

In [12]:
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (20,20))
for i in tqdm(range(0,5)):
              rand=np.random.randint(len(A))
              img=imread(A[rand])
              img=resize(img,(128,128))
              ax[i].imshow(img)

In [13]:
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (20,20))
for i in tqdm(range(0,5)):
              rand=np.random.randint(len(H))
              img=imread(H[rand])
              img=resize(img,(128,128))
              ax[i].imshow(img)

In [14]:
image=[]
label=[]
for i in tqdm(range(len(A))):
              img=imread(A[i])
              img=resize(img,(128,128))
              image.append(img)
              label.append(1)
              
for i in tqdm(range(len(H))):
              img_=imread(H[i])
              img=resize(img,(128,128))
              image.append(img)
              label.append(0)

image=np.array(image)
label=np.array(label)

In [15]:
del A 
del H

In [16]:
image, label = shuffle(image, label, random_state = 42)

In [17]:
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (20,20))
for i in tqdm(range(0,5)):
              rand=np.random.randint(len(image))
              ax[i].imshow(image[rand])
              a=label[rand]
              if a==1:
                ax[i].set_title('diseased')
              else:
                ax[i].set_title('fine')

In [18]:
X=image
y=label

In [19]:
del image
del label

In [20]:
X_val = []

for image_name in valid_data.new_names:
    # Loading images
    img = imread('../input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data/' + image_name)
    # Resizing 
    img = resize(img, (128,128))
    # Appending them into list
    X_val.append(img)
 
# Converting into array
X_val = np.array(X_val)


# Storing target values as well 
y_val = valid_data.labels.values

In [21]:
train_datagen  = ImageDataGenerator(horizontal_flip=True,
                                    vertical_flip=True,
                                    zoom_range = 0.2)

train_datagen.fit(X)

In [22]:
#Contribution: Made some changes in layers and add some hidden layers 

model=Sequential()
model.add(InputLayer(input_shape=(128,128,3)))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(Dense(4))
model.add(MaxPool2D(pool_size=(2,2),padding='valid'))
model.add(Dropout(.2))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2))
model.add(MaxPool2D(pool_size=(2,2),padding='valid'))
model.add(Dropout(.2))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))
model.add(MaxPool2D(pool_size=(2,2),padding='valid'))
model.add(Dropout(.2))

model.add(Conv2D(filters=256,kernel_size=(3,3),padding='valid',activation='relu')) #contribution
model.add(BatchNormalization())#contribution

model.add(Flatten())

model.add(Dense(units = 128, activation = 'relu')) #contribution
model.add(Dropout(0.3))

model.add(Dense(units = 64, activation = 'relu')) #contribution
model.add(Dropout(0.3))


model.add(Dense(units=1, activation='sigmoid'))


In [23]:
model.summary()

In [24]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])



In [25]:
filepath = './best_weights.hdf5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 15)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

callback_list = [earlystopping, checkpoint]

In [26]:
len(X),len(X_val)

In [27]:
#contribution
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 patience=10, 
                                 verbose=2, 
                                 factor=.75)


model_checkpoint= ModelCheckpoint("/best_result_checkpoint", monitor='val_loss', save_best_only=True, verbose=0)

In [28]:
#history1 = model.fit(train_datagen.flow(X, y, batch_size = 512),
          #validation_data = (X_val, y_val),
         # epochs = 2,
          #verbose = 1,
         # callbacks =[earlystopping])

In [29]:
#history2 = model.fit(train_datagen.flow(X, y, batch_size = 512),
         # validation_data = (X_val, y_val),
         # epochs = 4,
          #verbose = 1,
          #callbacks =[earlystopping])

In [ ]:
#contribution: Reduced in Epochs and changed in Activation function using Softmax  and reduce batch size 
history = model.fit(train_datagen.flow(X, y, batch_size = 212),
          validation_data = (X_val, y_val),
          epochs = 6,
          verbose = 1,
          callbacks =[earlystopping])

tf.keras.models.save_model(model,'my_model.hdf5')

In [ ]:
model.summary

In [ ]:
#my Contribution

print(history.history['accuracy'])

plt.plot(history.history['accuracy'],'--', label='accuracy on training set')
plt.plot(history.history['val_accuracy'], label='accuracy on validation set')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
#my Contribution

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
  
# creating the dataset
data = {'Reference Accuracy':65, 'My Accuracy':97}
courses = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Comparison Between My Accuracy and Reference Accuracy ")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
Reference:
    https://www.kaggle.com/dimaorizz/kravtsov-lab7
    https://towardsdatascience.com/cnn-architectures-a-deep-dive-a99441d18049
    https://ieeexplore.ieee.org/document/9071471
    google.com/leukemia classification
    